# Importações iniciais

In [1]:
import pandas as pd

In [2]:
ST = pd.read_csv("../data/estabelecimentos/STSC202202.csv",low_memory=False)

## Tipos de leito

In [3]:
ST.rename(columns={"QTLEITP1": "leito_cirurgico", "QTLEITP2": "leito_clinico", "QTLEITP3": "leito_complementar"},inplace=True)

## Agregando dados

## Leitos gerais

In [4]:
leitos_columns = [x for x in ST.columns if x.startswith("QTLEIT")]
leitos = ST[leitos_columns].apply(sum, axis=1)
ST["leitos"] = leitos
ST.drop(labels=leitos_columns,axis="columns",inplace=True)

## Consultórios

In [5]:
consultorios_columns = [x for x in ST.columns if x.startswith("QTINST")]
consultorios = ST[consultorios_columns].apply(sum, axis=1)
ST["consultorios"] = consultorios
ST.drop(labels=consultorios_columns,axis="columns",inplace=True)

## Metadados

In [6]:
atendimento_hospitalar = {
    "URGEMERG": "URGÊNCIA/EMERGÊNCIA",
    "ATENDAMB": "ATENDIMENTO AMBULATORIAL",
    "CENTRCIR": "ATENDIMENTO HOSPITALAR-CENTRO CIRÚRGICO",
    "CENTROBS": "ATENDIMENTO HOSPITALAR-CENTRO OBSTÉTRICO",
    "CENTRNEO": "ATENDIMENTO HOSPITALAR-UNIDADE NEONATAL",
    "ATENDHOS": "ATENDIMENTO HOSPITALAR"
}

tipos_atendimento = {
    "AP01": "INTERNACAO",
    "AP02": "AMBULATORIAL",
    "AP03": "SADT",
    "AP04": "URGENCIA",
    "AP05": "OUTROS",
    "AP06": "VIGILANCIA",
    "AP07": "REGULACAO"
}

tipos_convênio = {
    "CV01": "SUS",
    "CV02": "PARTICULAR",
    "CV03": "PLANO-SEGURO PROPRIO",
    "CV04": "PLANO-SEGURO TERCEIRO",
    "CV05": "PLANO de SAUDE PUBLICO",
    "CV06": "SAUDE PRIVADO"
}



In [7]:
ATENDIMENTOS_CONVENIOS = {}
for key1, val1 in tipos_atendimento.items():
    for key2, val2 in tipos_convênio.items():
        ATENDIMENTOS_CONVENIOS[key1+key2] = f"{val1} ({val2})"

In [9]:
for key, val in tipos_atendimento.items():
    colunas = [x for x in ST.columns if x.startswith(key)]
    ST["servico_" + val.lower()] = ST[colunas].apply(sum,axis=1)

In [10]:
programa = {
    "G1": "Atenção Básica Ambulatorial",
    "G2": "Média Complexidade Ambulatorial",
    "G3": "Cod 3",
    "G4": "Alta Complexidade Ambulatorial",
    "G5": "Média Complexidade Hospitalar",
    "G6": "Alta Complexidade Hospitalar"
}

PROGRAMAS = {}
for x, label in [("E", " (Estadual)"), ("M", " (Municipal)")]:
    for key, val in programa.items():
        PROGRAMAS["GESPR"+key+x] = val + label

# Iterando sobre o dataset

In [12]:
metadados = pd.DataFrame(columns=["CNES", "key", "value"], dtype="category")

In [13]:
def append_metadados(CNES, key, val):
    global metadados
    new_val = pd.Series([CNES, key, val], index=["CNES", "key", "value"])
    metadados = pd.concat([metadados, new_val.to_frame().T], ignore_index=True)

In [15]:
for i, reg in ST.iterrows(): 
    for key, val in ATENDIMENTOS_CONVENIOS.items():
        if reg[key] == 1:
            append_metadados(reg.CNES, "atendimentos", val)
    for key, val in PROGRAMAS.items():
        if reg[key] == 1:
            append_metadados(reg.CNES, "programa", val)
    for key, val in atendimento_hospitalar.items():
        if reg[key] == 1:
            append_metadados(reg.CNES, "hospital", val)

In [16]:
metadados.to_pickle("../parsed-data/metadados.pd.pkl")

In [17]:
interesse = ["CNES", "CODUFMUN", "COD_CEP",  "leito_cirurgico", "leito_clinico", "leito_complementar", "leitos", "consultorios" ]

In [18]:
ST[interesse].to_pickle("../parsed-data/dados.pd.pkl")